In [1]:
!pip install -q -U google-generativeai



In [2]:
import google.generativeai as genai
from google.colab import userdata


In [3]:
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))


In [4]:
list(genai.list_models())



[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
      

In [5]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content="Pakistan zinda bad we love our country",
    task_type="retrieval_document",
    title="Embedding of single string",
)

# # 1 input > 1 vector output
# print(str(result["embedding"])[:50], "... TRIMMED]")

result['embedding']

[-0.03838823,
 0.052146558,
 -0.07062306,
 -0.037940446,
 0.06602876,
 0.003412638,
 0.011060191,
 0.011297473,
 0.0088465065,
 0.049196165,
 0.026362207,
 0.029781424,
 0.103020616,
 0.0018105474,
 0.009687083,
 -0.1145385,
 0.053375162,
 0.027896093,
 -0.069912076,
 0.033948876,
 0.0021454412,
 -0.06642034,
 0.04420987,
 -0.021766845,
 -0.05236885,
 0.012519431,
 -0.006010968,
 -0.005167873,
 -0.0061494075,
 0.0031098027,
 0.025889847,
 0.056373067,
 0.046986956,
 -0.054916363,
 0.02060043,
 0.021911908,
 -0.028752202,
 0.022969386,
 0.0386514,
 -0.030484943,
 -0.089530766,
 0.015129875,
 -0.06397387,
 0.047075633,
 -0.013017894,
 -0.029871592,
 -0.0067695505,
 -0.0074492395,
 -0.0059240796,
 0.03697813,
 0.018429233,
 -0.001360107,
 -0.011460476,
 0.011616342,
 -0.025911184,
 -0.05487599,
 -0.036950245,
 -0.009039906,
 0.011936421,
 -0.002934238,
 -0.027310636,
 6.6082706e-05,
 -0.020024965,
 -0.027493374,
 0.002532368,
 -0.015001376,
 -0.076121554,
 0.032683495,
 -0.092314504,
 0.0

In [6]:
len(result['embedding'])


768

In [7]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would a woodchuck chuck?",
        "How does the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of list of strings",
)

# A list of inputs > A list of vectors output
for v in result["embedding"]:
    print(str(v)[:50], "... TRIMMED ...", len(v))

[-0.036453027, 0.033254996, -0.03970925, -0.002628 ... TRIMMED ... 768
[-0.01591948, 0.032582663, -0.081024624, -0.011298 ... TRIMMED ... 768
[0.00037063024, 0.03763057, -0.122695684, -0.00951 ... TRIMMED ... 768


# Building Vector Stores & Retreival using Chroma DB and Langchain


In [8]:
!pip install -Uq langchain-chroma


In [9]:

import getpass
import os


In [10]:

!pip install pypdf
!pip install langchain-community

# Load doc

In [11]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/Panaversity Cloud Native Applied Generative and Agentic AI Engineer (1).pdf")
pages = loader.load_and_split()
docs = loader.load()


# Split doc

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [13]:
!pip install -Uq langchain-google-genai


In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",
                                          google_api_key=userdata.get('GOOGLE_API_KEY'))

In [15]:
# embeddings.embed_query("What's our Q1 revenue?")

# Embd doc

In [16]:
from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents = splits,
    embedding=embeddings)

In [17]:
list(dir(vectorstore))


['_Chroma__ensure_collection',
 '_Chroma__query_collection',
 '_LANGCHAIN_DEFAULT_COLLECTION_NAME',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_chroma_collection',
 '_client',
 '_client_settings',
 '_collection',
 '_collection_metadata',
 '_collection_name',
 '_cosine_relevance_score_fn',
 '_embedding_function',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_persist_directory',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aa

In [18]:
vectorstore


In [19]:
vectorstore.similarity_search("python")


[Document(id='69c12aea-b181-4eea-a78b-5d865d47da1a', metadata={'page': 13, 'page_label': '14', 'source': '/content/Panaversity Cloud Native Applied Generative and Agentic AI Engineer (1).pdf'}, page_content="5.Whydon'tweuseTypeScript(Node.js)todevelopAPIsinsteadofusingPython?WewillnotuseTypescriptinGenAIAPIdevelopmentbecausePythonisaprioritywiththeAIcommunitywhenworkingwithAIandifanyupdatescomeinlibrariestheywillfirstcomeforPython.PythonisalwaysabetterchoicewhendealingwithAIandAPI.\n● PythonisthedefactostandardforAIDevelopment.● TypeScriptisamoremodernlanguagethatisgainingpopularityforWebDevelopment,butPythonismorewidelyusedandhasalargerecosystemoflibrariesandframeworksavailable,especiallyforAI.● TypeScriptisusedforwebuserinterfaces,whilePythonisusedforAPIs.● Inthesecondquarter, studentswilllearntodevelopAPIsusingPythoninsteadofTypeScript.● PythonisamorecommonlyusedlanguageforAIandAPIdevelopment,andithasalargerecosystemoflibrariesandframeworksavailableforthesepurposes.● TypeScriptisamo

In [20]:
await vectorstore.asimilarity_search("cloud")


[Document(id='47f7aa46-99ac-4bd0-a082-d8b989bd5bbe', metadata={'page': 5, 'page_label': '6', 'source': '/content/Panaversity Cloud Native Applied Generative and Agentic AI Engineer (1).pdf'}, page_content="Amazonisstillthecloudkingbasedonmarketshare.Butmanyanalystsagree:Inthebattleforthecloud,AIisnowagame-changer—andAmazon'smaincompetitors,particularlyMicrosoft,havethemomentum.\nInourprogramwewillbeusingAzureasourdefaultproviderforteachinganddeployment.Wewillbeusingusingtheseservices:\nGetafreeAzureAccountnow:https://azure.microsoft.com/en-us/freeNote:UseGitHubAccounttostartanAzurefreetrial\nAzureContainerApps(WewillStartfromthisserviceusingDaprandKeda)https://azure.microsoft.com/en-us/products/container-appsGetstartedwiththefreetier:Thefirst180,000vCPUpersecond,360,000GiB/s,and2millionrequestseachmontharefree.Watch:https://www.youtube.com/watch?v=0HwQfsa03K8Deploy:https://learn.microsoft.com/en-us/azure/container-apps/code-to-cloud-options\nAzureContainerRegistryhttps://azure.microsof

In [21]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("langchain")

[(Document(id='e3690a43-1e88-49ba-8be1-f8efbbc36e94', metadata={'page': 14, 'page_label': '15', 'source': '/content/Panaversity Cloud Native Applied Generative and Agentic AI Engineer (1).pdf'}, page_content='LangChainisaplatformthatallowsyoutointeractwithvariouslanguagemodelsfromdifferentproviderssuchasOpenAI,GoogleGemini,HuggingFaceTransformers,etc.YoucanuseLangChaintocreateapplicationsthatleveragethepowerofnaturallanguageprocessingwithouthavingtodealwiththecomplexityofAPIsorSDKs.LangChainprovidesauser-friendlyinterfacethatletsyouchoosethemodelyouwanttouse,customizetheparametersyouwanttoapply, andseetheresultsinreal-time.\n15'),
  0.45153141021728516),
 (Document(id='15c83ad5-2087-4973-9ae1-85408229b7d6', metadata={'page': 14, 'page_label': '15', 'source': '/content/Panaversity Cloud Native Applied Generative and Agentic AI Engineer (1).pdf'}, page_content='6.WhatisthedifferencebetweenOpenAICompletionAPI,OpenAIAssistantAPI,GoogleGeminiMulti-ModalAPI,andLangChain?ThedifferencebetweenO

# Retrievers

In [22]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["python"])

[[Document(id='69c12aea-b181-4eea-a78b-5d865d47da1a', metadata={'page': 13, 'page_label': '14', 'source': '/content/Panaversity Cloud Native Applied Generative and Agentic AI Engineer (1).pdf'}, page_content="5.Whydon'tweuseTypeScript(Node.js)todevelopAPIsinsteadofusingPython?WewillnotuseTypescriptinGenAIAPIdevelopmentbecausePythonisaprioritywiththeAIcommunitywhenworkingwithAIandifanyupdatescomeinlibrariestheywillfirstcomeforPython.PythonisalwaysabetterchoicewhendealingwithAIandAPI.\n● PythonisthedefactostandardforAIDevelopment.● TypeScriptisamoremodernlanguagethatisgainingpopularityforWebDevelopment,butPythonismorewidelyusedandhasalargerecosystemoflibrariesandframeworksavailable,especiallyforAI.● TypeScriptisusedforwebuserinterfaces,whilePythonisusedforAPIs.● Inthesecondquarter, studentswilllearntodevelopAPIsusingPythoninsteadofTypeScript.● PythonisamorecommonlyusedlanguageforAIandAPIdevelopment,andithasalargerecosystemoflibrariesandframeworksavailableforthesepurposes.● TypeScriptisam

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = userdata.get('GOOGLE_API_KEY')
)


In [24]:
# llm.invoke("tell about cats?")


In [25]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

In [26]:
# Augment
prompt = ChatPromptTemplate.from_messages([("human", message)])


# RAG

In [27]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm


In [28]:
response = rag_chain.invoke("tell about quater 3 ")

print(response.content)



In the third quarter (Quarter 3) of the Engineering Program, students learn to use Python-based FastAPI as a core library for API development.  This will give them the skills to create powerful and efficient software applications to solve various business problems.


# Click on below link for documentation
[Documentation click here](https://python.langchain.com/api_reference/google_genai/embeddings/langchain_google_genai.embeddings.GoogleGenerativeAIEmbeddings.html#langchain_google_genai.embeddings.GoogleGenerativeAIEmbeddings)



# Face detection with embedding


In [29]:
!pip install -Uq facenet-pytorch


In [30]:
!pip install torch==2.5.1+cu121 torchvision==0.16.1+cu121 torchaudio==2.5.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==2.5.1+cu121 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1.2+rocm5.6, 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.0

In [31]:
!pip install -Uq pillow


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 11.1.0 which is incompatible.


In [32]:
import torch
import torch.nn as nn
import torchvision.models as models

import torchvision.transforms as transforms
from PIL import Image

In [33]:
from facenet_pytorch import MTCNN, InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2').eval()

In [34]:
model

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

# Preprocessing function to transform the image into a tensor


In [35]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Function to create image embeddings
def create_image_embedding(image_path):
    try:
        input_tensor = preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)# ebedding important line
        return embeddings.squeeze().numpy()
    except Exception as e:
        print("Error:", e)
        return None

In [36]:
!mkdir images



mkdir: cannot create directory ‘images’: File exists


# prompt: create python function where we provide image url and imag_name then it save in images folder


In [37]:
import requests
import os

def save_image_from_url(image_url, image_name):
  """
  Downloads an image from a URL and saves it to the 'images' folder.

  Args:
    image_url: The URL of the image to download.
    image_name: The name of the file to save the image as.
  """
  try:
    if not os.path.exists("images"):
      os.makedirs("images")

    image_path = os.path.join("images", image_name)

    response = requests.get(image_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(image_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image saved to: {image_path}")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")

In [38]:
save_image_from_url("https://media.licdn.com/dms/image/v2/D4E03AQEEn9DuNlQwvw/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1664654245747?e=2147483647&v=beta&t=NGB0a9aqsgdyxpbuO3rqws95ogJnL_6aRtBDS7IWPfw","s1.jpg")
save_image_from_url("https://avatars.githubusercontent.com/u/10209765?v=4", "q1.jpg")

save_image_from_url("https://media.licdn.com/dms/image/v2/D4D22AQFmuEiR8ttUmw/feedshare-shrink_800/feedshare-shrink_800/0/1711203894556?e=2147483647&v=beta&t=GEZGp_cdogNJCJIGidoEw_DjW2FXZcG4nUUlaNF1Zlc","z1.jpg")
save_image_from_url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBBiqefc7Le97Rn0udVVBkur7RlU53FcQh1A&s",'z2.jpg')
save_image_from_url("https://scontent.fkhi4-4.fna.fbcdn.net/v/t39.30808-6/468785380_10160566910882765_300507882801991935_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=6ee11a&_nc_eui2=AeEk77SJKagGymTo3ibNnnx9YsjCm8DJ0lRiyMKbwMnSVMJqs7YWsJDuzKzXyLHLoFk&_nc_ohc=QJMm9K-AE4QQ7kNvgFE0N2o&_nc_oc=Adi1r8eogMcuDIMMLJvliCOnaaXQ2KnUbbJvY94aAnfInkDB-fyB_1ZXBpDQnWTkZnY&_nc_zt=23&_nc_ht=scontent.fkhi4-4.fna&_nc_gid=AcSynbwc6ukNTxWnUzjhtEe&oh=00_AYCvnF-vj63T-X69PZgBk6JvVjepzybVukgHPSM_6BXkGQ&oe=678003A4",'s2.jpg')
save_image_from_url("https://i.ytimg.com/vi/7QD3GKvSyMk/hqdefault.jpg?sqp=-oaymwEmCOADEOgC8quKqQMa8AEB-AHOBYAC0AWKAgwIABABGGUgXChPMA8=&rs=AOn4CLB2EaZsLrClGHqUMUhApQ_sxAcF7Q","q2.jpg")


Image saved to: images/s1.jpg
Image saved to: images/q1.jpg
Image saved to: images/z1.jpg
Image saved to: images/z2.jpg
Error downloading image: 403 Client Error: Forbidden for url: https://scontent.fkhi4-4.fna.fbcdn.net/v/t39.30808-6/468785380_10160566910882765_300507882801991935_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=6ee11a&_nc_eui2=AeEk77SJKagGymTo3ibNnnx9YsjCm8DJ0lRiyMKbwMnSVMJqs7YWsJDuzKzXyLHLoFk&_nc_ohc=QJMm9K-AE4QQ7kNvgFE0N2o&_nc_oc=Adi1r8eogMcuDIMMLJvliCOnaaXQ2KnUbbJvY94aAnfInkDB-fyB_1ZXBpDQnWTkZnY&_nc_zt=23&_nc_ht=scontent.fkhi4-4.fna&_nc_gid=AcSynbwc6ukNTxWnUzjhtEe&oh=00_AYCvnF-vj63T-X69PZgBk6JvVjepzybVukgHPSM_6BXkGQ&oe=678003A4
Image saved to: images/q2.jpg


# Embedding of image q2

In [39]:
# Example usage
image_path = "./images/q2.jpg"
q2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", q2.shape)
print("Image Embedding:", q2)

Image Embedding Shape: (512,)
Image Embedding: [ 0.00058278 -0.01463297 -0.10330155  0.03984535  0.08525636  0.08029363
 -0.01011138  0.04867227 -0.00259888 -0.0077198  -0.0331905   0.06225294
  0.01708761 -0.00982595 -0.01638006  0.00054205  0.04426373  0.00519726
  0.04448376 -0.08979201 -0.06033407 -0.00162232  0.09186839  0.03262272
  0.06873461  0.02532319  0.03512866 -0.05809444  0.00151727  0.05172818
 -0.04107905 -0.03755797 -0.04134395 -0.00761167 -0.00674731  0.04443471
  0.00570058 -0.02845279 -0.12002273  0.05528227  0.00132485  0.00437373
 -0.02495593  0.01842853 -0.00357915  0.01889837  0.02980521  0.10357354
 -0.11238679 -0.02633037  0.02794239 -0.0152881   0.04015452 -0.0039458
 -0.09202526  0.0715652  -0.03382103  0.07384298  0.01921727 -0.04263638
  0.02758526  0.03841295 -0.04435986 -0.05347932 -0.04511738  0.03700717
 -0.00194125 -0.00450149  0.03157888  0.02796174  0.02416223  0.01202647
 -0.00905218  0.00880154  0.0189873  -0.01363976 -0.0499587  -0.01557556
  0.0

In [40]:
image_path = "/content/images/s1.jpg"
cat2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", cat2.shape)
print("Image Embedding:", cat2)


Image Embedding Shape: (512,)
Image Embedding: [ 7.76378484e-03  1.38627877e-02  3.42880785e-02  5.59410453e-02
  4.43015993e-02  4.22530621e-02  3.66532840e-02  9.78047494e-03
 -1.34703349e-02  1.68228522e-02  8.26280266e-02  7.83666130e-03
  4.87103760e-02 -3.90704256e-03  1.06615685e-02 -7.18539953e-02
 -4.96409610e-02  1.04582205e-01  1.67736411e-02  2.15226728e-02
 -2.09242180e-02  8.07328224e-02  1.67470835e-02 -1.14253521e-01
 -4.25868072e-02 -2.27696840e-02 -1.50764631e-02 -4.02070023e-02
 -2.33913157e-02 -4.37326618e-02  7.05495998e-02  7.63667449e-02
  5.58592677e-02  1.28479674e-02  3.95264030e-02 -4.74360399e-02
  1.70672743e-03  1.20020993e-02 -4.78693005e-03 -1.22949760e-02
  1.39081292e-02 -2.78096441e-02  2.48735808e-02 -2.39850115e-03
 -1.18011301e-02  3.69508145e-03 -4.81235720e-02  1.29993325e-02
 -6.90511912e-02 -8.01179186e-02 -2.32403465e-02 -6.55335188e-02
  3.54257748e-02  5.77532388e-02  1.61157586e-02 -5.35045713e-02
  3.46317366e-02  4.95921448e-02 -6.5778270

In [41]:
q1 = create_image_embedding("./images/q1.jpg")
q2 = create_image_embedding("./images/q2.jpg")
s1 = create_image_embedding("./images/s1.jpg")
z1 = create_image_embedding("./images/z1.jpg")
z2 = create_image_embedding("./images/z2.jpg")

# Save image embedding into vector database

In [42]:

!pip install -U milvus-lite

!pip install -U pymilvus

In [43]:
from pymilvus import MilvusClient
client = MilvusClient("./milvus_demo.db")


In [44]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="images",
    dimension=512  # The vectors we will use in this demo has 384 dimensions
)


In [46]:

data = [
    {"id": 1, "person_name": "Qasim", "vector": q1},
    {"id": 2, "person_name": "Qasim", "vector": q2},
    {"id": 3, "person_name": "Shahzad", "vector": s1},
    {"id": 5, "person_name": "Zia Khan", "vector": z1},
    {"id": 6, "person_name": "Zia Khan", "vector": z2}
]


In [47]:

res = client.insert(
    collection_name="images",
    data=data
)

In [48]:
res = client.search(
    collection_name="images",
    data=[s1],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)


data: ["[{'id': 3, 'distance': 1.0, 'entity': {'person_name': 'Shahzad', 'id': 3}}]"]
